<font color='black' size='5'>To be added:</font>
1. Data visualization for default features.
1. inferences from that data.
1. New features based on those inferences (Partially done).
1. Specified preprocessing for the various features.

using the nbextention "Initialization cells", I first of all save the standard output to the notebook, as it changes with each kernel. <br>
The condition makes sure that no matter what happens, as long as I don't go out of my way to screw up, `saved_std` is always the original standard output.  
This was done so that I can output the verbose cross-validation grids to the terminal without bogging down this notebook.

In [8]:
import sys

if str(type(sys.stdout)) == "<class 'ipykernel.iostream.OutStream'>":
    saved_std = sys.stdout
else:
    sys.stdout = saved_std

import warnings
warnings.filterwarnings('ignore') ## Not sure what this does anymore :(
sys.stdout

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import itertools
from math import ceil

In [5]:
'''
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier,
                              AdaBoostClassifier,ExtraTreesClassifier, VotingClassifier)
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn import metrics
from scipy.stats import uniform
'''

print("These are kept here for now. This used to be both the modeling and preprocessing notebook.")

These are kept here for now. This used to be both the modeling and preprocessing notebook.


In [259]:
df = pd.read_csv('train.csv', index_col='PassengerId').fillna(value={'Cabin':'Unknown'})
test = pd.read_csv('test.csv', index_col='PassengerId').fillna(value={'Cabin':'Unknown'})
y = df['Survived']
df.drop(['Survived'],axis=1,inplace=True)
df

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S
...,...,...,...,...,...,...,...,...,...,...
887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,Unknown,S
888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,Unknown,S


Note that only people with a Pclass of 1 have a cabin. Which makes sense since they are the richest.

As shown in this __[forum dicussion](https://www.kaggle.com/c/titanic/discussion/57447)__ the surname contains a lot of information. <br>While it may not generalize, it never hurts to try :).

In [260]:
df['Surname'] =  [Name.split(',')[0] for Name in df['Name']]
test['Surname'] =  [Name.split(',')[0] for Name in test['Name']]
len(df['Surname'].unique())

667

There are many unique surnames and so, instead of the usual OneHot encoding, I'll be using the Binary encoding (Same thing goes for tickets).  

<font color='red' size='6'>May Change</font>

Another feature which may be very useful is the person's title, as it contains both gender and economic status.<br>
The method by which I find all possible titles and assign each person a title is taken from __[here](https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/)__.

In [261]:
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if big_string.find(substring) != -1:
            return substring
    print(big_string)
    return None

In [262]:
# Titles = ['Mr','Mrs','Miss','Master'] 

# # all other titles must be found and entered by hand
# # since there is no pattern to their placement
# for Name in df['Name']:
#     if substrings_in_string(Name,Titles) == None:
#         Title = input("Please enter this person's title:")
#         Titles.append(Title)

# for Name in test['Name']:
#     if substrings_in_string(Name,Titles) == None:
#         Title = input("Please enter this person's title:")
#         Titles.append(Title)

Titles = ['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme',
          'Ms', 'Major', 'Mlle', 'Col', 'Capt', 'Countess', 'Jonkheer']


df['Title'] = df['Name'].apply(substrings_in_string, substrings=Titles)
test['Title'] = test['Name'].apply(substrings_in_string, substrings=Titles)

The list of titles is quite big and contains many titles that appear with a very low frequency.<br>
I'd like to simplify them, the method by which I reduce the number of titles is also taken from __[here](https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/)__.

In [263]:
def replace_titles(x):
    title=x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
        return 'Mr'
    elif title in ['Countess', 'Mme']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title

In [264]:
df['Title'] = df.apply(replace_titles, axis=1)
test['Title'] = test.apply(replace_titles, axis=1)

The deck which a person stayed on may be relevant, as some cabins were closer to the lifeboats.<br>
However, this column contains by far the highest percentage of missing data which may hurt the model or not affect it at all.

In [265]:
df['Deck'] = [Cab[0] for Cab in df['Cabin']]
test['Deck'] = [Cab[0] for Cab in test['Cabin']]
df

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Deck
PassengerId,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S,Braund,Mr,U
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Cumings,Mr,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S,Heikkinen,Miss,U
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Futrelle,Mr,C
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S,Allen,Mr,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,Unknown,S,Montvila,Mr,U
888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Graham,Miss,B
889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,Unknown,S,Johnston,Miss,U


We'll handle different data types differently in our preprocessing.<br>
The pipelines help adding other transformers later and keeping things consistent.<br>
The binary columns don't need an Imputer since they don't have any missing data in both datasets.

In [266]:
Numerical_Pre = make_pipeline(SimpleImputer(strategy='mean'))
OneHot_Pre = make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder(handle_unknown = 'ignore'))
Binary_Pre = make_pipeline(BinaryEncoder())
Col_Num = ['Age','Fare','SibSp','Parch']
Col_OHE = ['Sex','Deck','Title','Embarked','Pclass']
Col_Bin = ['Surname','Ticket']


Preprocess = ColumnTransformer([('Col_OHE', OneHot_Pre, Col_OHE),
                                ('Col_Bin', Binary_Pre, Col_Bin),
                                ('Col_Num', Numerical_Pre, Col_Num),
                                ])

In [267]:
PreProOHE = [sorted([str(Col) + "_" + str(i) for i in df[Col].unique() if str(i) != 'nan']) for Col in Col_OHE]
PreProBin = [[str(Col) + "_" + str(i+1) for i in range(ceil(np.log2(len(df[Col].unique()))) + 1)] for Col in Col_Bin]
Cols = list(itertools.chain(*(PreProOHE+PreProBin))) + Col_Num
Cols

['Sex_female',
 'Sex_male',
 'Deck_A',
 'Deck_B',
 'Deck_C',
 'Deck_D',
 'Deck_E',
 'Deck_F',
 'Deck_G',
 'Deck_T',
 'Deck_U',
 'Title_Master',
 'Title_Miss',
 'Title_Mr',
 'Title_Mrs',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S',
 'Pclass_1',
 'Pclass_2',
 'Pclass_3',
 'Surname_1',
 'Surname_2',
 'Surname_3',
 'Surname_4',
 'Surname_5',
 'Surname_6',
 'Surname_7',
 'Surname_8',
 'Surname_9',
 'Surname_10',
 'Surname_11',
 'Ticket_1',
 'Ticket_2',
 'Ticket_3',
 'Ticket_4',
 'Ticket_5',
 'Ticket_6',
 'Ticket_7',
 'Ticket_8',
 'Ticket_9',
 'Ticket_10',
 'Ticket_11',
 'Age',
 'Fare',
 'SibSp',
 'Parch']

All of this was done so we could understand the preprocessing and data we're fitting better.

# Comparing models:
We'll look at a quick estimation of the best hyper parameters for each model and the accuracy score.  

<font color='red' size='6'>Changes:</font>  
This section will no longer be part of this notebook and the last section will be extended.

In [268]:
def print_best(Grid,Prints = True):
    if Prints:
        print(Grid.best_score_,Grid.best_params_)
    if not Prints:
        return [Grid.best_score_,Grid.best_params_]

In [211]:
Gradient = GradientBoostingClassifier(warm_start=True)
Grad_Pipe = Pipeline([('Preprocess', Preprocess),
                     ('Gradient', Gradient)])

params_Grad = dict(Gradient__n_estimators = [200,300],
                   Gradient__max_depth=[3,5], 
                   Gradient__random_state=[12],
                   Gradient__learning_rate=[0.1,0.15])

In [226]:
Rand_Grad = RandomizedSearchCV(Grad_Pipe,params_Grad,cv=5,verbose=10,scoring='accuracy',n_iter=4,random_state=1,error_score='raise')

sys.stdout = open(1, 'w')
Rand_Grad.fit(df,y)
print('Done with grid :)')
sys.stdout = saved_std
print_best(Rand_Grad)


0.8249639068482832 {'Gradient__random_state': 12, 'Gradient__n_estimators': 300, 'Gradient__max_depth': 3, 'Gradient__learning_rate': 0.1}


In [227]:
xgbtree = XGBClassifier(use_label_encoder=False, eval_metric='rmse', objective='binary:logistic')
xgblin = XGBClassifier(booster = 'gblinear',use_label_encoder=False, eval_metric='rmse', objective='binary:logistic')

Pipe_tree = Pipeline([('Preprocess', Preprocess),
                     ('tree', xgbtree)])

Pipe_lin = Pipeline([('Preprocess', Preprocess),
                     ('lin', xgblin)])

xgb_distribution1 = dict(tree__eta = uniform(), tree__gamma = uniform(0,10),
                        tree__max_depth = [3,6,9])
xgb_distribution2 = dict(lin__eta = uniform())

Rand_xgtree = RandomizedSearchCV(Pipe_tree,xgb_distribution1,scoring='accuracy',cv=5,verbose=10,n_iter=10,random_state=1)
Rand_xglin = RandomizedSearchCV(Pipe_lin,xgb_distribution2,scoring='accuracy',cv=5,verbose=10,n_iter=10,random_state=1)

sys.stdout = open(1, 'w')
Rand_xgtree.fit(df,y)
print('Done with grid :)')
sys.stdout = saved_std
print_best(Rand_xgtree)

sys.stdout = open(1, 'w')
Rand_xglin.fit(df,y)
print('Done with grid :)')
sys.stdout = saved_std
print_best(Rand_xglin)

0.821542903772519 {'tree__eta': 0.3965807272960261, 'tree__gamma': 3.8791074116200743, 'tree__max_depth': 3}
0.8137467830016949 {'lin__eta': 0.1862602113776709}


In [269]:
SVC = Pipeline([('Preprocess',Preprocess),('SVC',SVC())])

SVC_distribution = dict(SVC__C=uniform(0,4), SVC__kernel = ['rbf','poly'])

Rand_SVC = RandomizedSearchCV(SVC,SVC_distribution,scoring='accuracy',cv=5,verbose=10,n_iter=25,random_state=1)

sys.stdout = open(1, 'w')
Rand_SVC.fit(df,y)
print('Done with grid :)')
sys.stdout = saved_std
print_best(Rand_SVC)

0.6970686083736112 {'SVC__C': 3.5055566091841532, 'SVC__kernel': 'rbf'}


In [270]:
KNN = Pipeline([('Preprocess',Preprocess),('KNN',KNeighborsClassifier())])
KNN_distribution = dict(KNN__n_neighbors = range(2,7))
Rand_KNN = RandomizedSearchCV(KNN,KNN_distribution,scoring='accuracy',cv=5,verbose=10,random_state=1)

sys.stdout = open(1, 'w')
Rand_KNN.fit(df,y)
print('Done with grid :)')
sys.stdout = saved_std
print_best(Rand_KNN)

0.6936664365074383 {'KNN__n_neighbors': 5}
